In [10]:
%pip install --user gym
%pip install pygame --pre
%pip install gym[box2d]
%pip install numpy
%pip install -U matplotlib
%pip install pynput
%pip install keyboard


import gym
from gym import wrappers
import matplotlib
import matplotlib.pyplot as plt
import keyboard
import time
import datetime as dt
from datetime import datetime
from Box2D import b2Vec2
import math
    
main_loop_time = 0.03 # measured in seconds

keys_being_pressed = {"w": False, "a": False, "s": False, "d": False}

def choosePlayerActionDiscrete(): # choose ONE action based on keys_being_pressed 
    
    if keys_being_pressed['s']: # brake pressed
        return 4 # brake
    elif keys_being_pressed['w']: # brake off, gas pressed
        return 3 # gas
    else: #just steering 
        if (keys_being_pressed['a'] and keys_being_pressed['d']) or not(keys_being_pressed['a'] or keys_being_pressed['d']): # both or neither
            return 0 # do nothing, steering cancels out
        elif keys_being_pressed['a']: # turn left ... these are backwards???
            return 2
        else: # turn right
            return 1


#target env, will create env object ok
env = gym.make("CarRacing-v2", render_mode='human', continuous=False)
env = gym.make('CarRacing-v2', render_mode='human', continuous=False, domain_randomize=False)

# If continuous = False, actions are:
# 0: do nothing
# 1: steer left   (pretty sure this is actually steer right and vice versa)
# 2: steer right
# 3: gas
# 4: brake

# If continuous = True, actions are:
# of type numpy.ndarray[] (length 3) 
# three floats: 
# index 0: steering, -1 is full left, +1 is full right
# index 1: gas
# index 2: braking

env.reset(seed=3)

end_loop = False
max_iterations = 100000 #aaa
i = 0

actions = ""
positions = []
avg_pos = b2Vec2(50, 0)
angles = [x for x in range(-180, 180)] #+180 for loop indices
xs = []
ys = []
inner_loop = [147.52, 148.02, 148.45, 148.91, 149.4, 149.85, 150.35, 150.77, 151.06, 151.46, 151.87, 152.29, 152.5, 152.59, 152.68, 152.82, 153.02, 153.26, 153.54, 153.89, 154.26, 154.71, 155.21, 155.71, 156.27, 156.86, 157.46, 158.22, 158.95, 159.73, 160.56, 161.44, 162.46, 163.53, 164.55, 165.73, 166.95, 168.23, 169.69, 171.14, 172.68, 173.28, 172.82, 170.85, 166.85, 160.7, 153.58, 143.33, 134.88, 128.16, 122.42, 117.23, 112.59, 108.24, 104.18, 100.66, 97.17, 93.95, 91.01, 88.34, 85.94, 83.57, 81.22, 79.13, 77.29, 75.47, 73.68, 71.92, 70.39, 69.1, 67.62, 66.38, 65.51, 64.74, 63.97, 63.32, 62.9, 62.56, 62.33, 62.17, 62.1, 62.11, 62.33, 62.75, 63.06, 63.39, 63.74, 64.21, 64.61, 65.13, 65.56, 66.12, 66.66, 66.97, 67.44, 67.84, 68.34, 68.87, 69.42, 70.0, 70.51, 71.22, 71.85, 72.54, 73.21, 73.92, 74.65, 75.58, 76.39, 77.21, 78.08, 79.22, 80.15, 81.35, 82.34, 83.64, 84.69, 86.04, 87.42, 89.1, 90.54, 92.34, 94.16, 96.0, 98.18, 100.08, 102.31, 104.9, 107.2, 109.86, 112.89, 115.96, 119.04, 122.48, 125.98, 130.22, 134.14, 138.82, 143.91, 149.02, 154.54, 160.84, 167.74, 175.04, 179.63, 181.08, 181.12, 179.86, 178.71, 177.5, 176.34, 175.21, 174.07, 173.09, 172.15, 171.27, 170.43, 169.65, 168.91, 168.24, 167.61, 167.04, 166.52, 166.11, 165.69, 165.38, 165.07, 164.81, 164.64, 164.49, 164.41, 164.37, 164.38, 164.44, 164.55, 164.72, 164.94, 165.2, 165.29, 165.76, 165.11, 163.04, 158.56, 153.7, 149.21, 144.76, 140.87, 137.02, 133.55, 130.11, 127.04, 124.17, 121.34, 118.7, 116.25, 113.84, 111.76, 109.57, 107.7, 105.72, 103.91, 102.28, 100.67, 99.09, 97.68, 96.29, 94.94, 93.73, 92.55, 91.4, 90.28, 89.3, 88.34, 87.41, 86.5, 85.62, 84.87, 84.04, 83.34, 82.66, 82.0, 81.37, 80.84, 80.26, 79.71, 79.25, 78.81, 78.39, 77.95, 77.58, 77.24, 76.91, 76.62, 76.38, 76.13, 75.91, 75.71, 75.54, 75.39, 75.27, 75.17, 75.1, 75.04, 75.01, 75.0, 75.02, 75.05, 75.11, 75.19, 75.31, 75.43, 75.6, 75.76, 75.95, 76.19, 76.42, 76.72, 76.98, 77.33, 77.71, 78.11, 78.46, 78.91, 79.38, 79.87, 80.39, 80.86, 81.36, 81.71, 81.93, 82.25, 82.54, 82.83, 83.02, 83.07, 83.04, 82.87, 82.81, 82.75, 82.64, 82.55, 82.44, 82.29, 82.14, 81.98, 81.82, 81.66, 81.35, 81.12, 80.91, 80.71, 80.55, 80.4, 80.28, 80.19, 80.12, 80.08, 80.06, 80.06, 80.09, 80.14, 80.21, 80.31, 80.42, 80.57, 80.73, 80.89, 81.05, 81.24, 81.45, 81.68, 81.94, 82.22, 82.57, 82.9, 83.26, 83.7, 84.1, 84.53, 85.01, 85.54, 86.01, 86.58, 87.14, 87.7, 88.22, 88.75, 89.32, 89.9, 90.52, 91.16, 91.92, 92.62, 93.44, 94.29, 95.16, 96.01, 96.79, 97.64, 98.5, 99.39, 100.43, 101.5, 102.62, 103.85, 105.54, 107.36, 110.46, 113.6, 117.97, 124.7, 136.05, 141.38, 143.04, 143.81, 144.69, 145.48, 145.99, 146.49, 146.98]
#inner_loop = [-1 for x in range(360)]
outer_loop = [165.63, 166.1, 166.61, 167.1, 167.63, 168.13, 168.65, 169.22, 169.72, 170.14, 170.51, 170.66, 170.87, 171.05, 171.16, 171.33, 171.53, 171.77, 172.09, 172.44, 172.76, 173.19, 173.62, 174.15, 174.73, 175.31, 176.01, 176.76, 177.56, 178.41, 179.41, 180.37, 181.39, 182.56, 183.78, 185.07, 186.41, 187.8, 189.25, 190.88, 192.06, 192.64, 192.74, 192.35, 191.17, 189.78, 188.13, 185.53, 182.46, 178.63, 173.84, 168.0, 161.74, 153.75, 146.67, 140.51, 135.61, 131.05, 126.83, 123.17, 119.55, 116.22, 113.18, 110.17, 107.45, 104.76, 102.35, 100.26, 98.18, 96.1, 94.27, 92.47, 90.69, 88.88, 87.37, 85.89, 84.44, 83.35, 82.51, 82.19, 82.14, 82.29, 82.34, 82.46, 82.64, 82.85, 83.07, 83.36, 83.73, 84.04, 84.48, 84.79, 85.26, 85.81, 86.27, 86.78, 87.33, 87.94, 88.59, 89.28, 90.01, 90.89, 92.08, 92.9, 93.61, 94.37, 95.5, 96.35, 97.22, 98.44, 99.7, 100.67, 102.07, 103.84, 105.3, 106.78, 108.22, 110.08, 111.63, 113.63, 115.66, 118.15, 120.26, 122.8, 125.27, 128.12, 130.73, 133.96, 136.94, 140.26, 143.92, 147.63, 151.69, 156.12, 160.59, 165.76, 170.97, 176.9, 182.88, 189.57, 194.91, 197.61, 199.29, 200.67, 201.43, 201.7, 201.05, 200.06, 198.99, 197.76, 196.32, 194.97, 193.72, 192.44, 191.26, 190.27, 189.17, 188.15, 187.32, 186.57, 185.76, 185.12, 184.48, 184.04, 183.62, 183.27, 183.0, 182.74, 182.57, 182.43, 182.36, 182.35, 182.38, 182.47, 182.62, 182.81, 183.09, 183.38, 183.77, 183.95, 184.07, 184.15, 184.17, 183.6, 181.74, 179.36, 176.41, 174.63, 166.98, 162.64, 158.59, 154.74, 151.08, 147.76, 144.64, 141.57, 138.81, 136.21, 133.66, 131.28, 129.06, 126.85, 124.79, 122.92, 121.1, 119.71, 118.23, 116.85, 115.39, 114.04, 112.82, 111.54, 110.43, 109.21, 108.12, 107.07, 106.04, 105.16, 104.19, 103.37, 102.57, 101.81, 101.03, 100.4, 99.69, 99.12, 98.48, 97.96, 97.48, 97.01, 96.58, 96.18, 95.8, 95.46, 95.15, 94.86, 94.61, 94.38, 94.22, 94.05, 93.92, 93.82, 93.75, 93.71, 93.69, 93.71, 93.75, 93.82, 93.91, 94.04, 94.21, 94.4, 94.63, 94.84, 95.13, 95.44, 95.78, 96.15, 96.55, 96.98, 97.44, 98.01, 98.5, 98.98, 99.44, 99.88, 100.16, 100.51, 100.87, 101.27, 101.55, 101.85, 102.16, 102.42, 102.4, 102.38, 102.3, 102.19, 101.96, 101.69, 101.47, 101.24, 100.96, 100.72, 100.3, 99.99, 99.71, 99.41, 99.14, 98.91, 98.63, 98.42, 98.23, 98.06, 97.91, 97.81, 97.73, 97.68, 97.66, 97.66, 97.7, 97.75, 97.79, 97.87, 97.98, 98.11, 98.29, 98.47, 98.72, 98.96, 99.27, 99.55, 99.93, 100.27, 100.71, 101.18, 101.68, 102.09, 102.62, 103.12, 103.61, 104.08, 104.61, 105.17, 105.75, 106.37, 107.0, 107.63, 108.39, 109.06, 109.87, 110.6, 111.46, 112.19, 113.04, 113.94, 114.87, 115.93, 117.38, 119.13, 121.49, 124.9, 129.01, 134.14, 140.86, 148.19, 151.61, 153.96, 155.89, 157.66, 159.13, 160.6, 161.25, 161.94, 162.64, 163.27, 163.91, 164.46, 164.85, 165.21]

while (i < max_iterations and not end_loop): #main gameplay loop
    i += 1
    
    for key in ['w', 'a', 's', 'd']:
        keys_being_pressed[key] = keyboard.is_pressed(key)

    if (keyboard.is_pressed('q')):
        end_loop = True

    start_time = datetime.now()
    target_end_time = start_time + dt.timedelta(seconds=main_loop_time)
    
    while datetime.now() < target_end_time: # wait for loop time
        pass

    #print("car pos: "  + str(env.car.hull.position))
    vector_to_center = env.car.hull.position - avg_pos
    #print(vector_to_center)
    rad = math.atan2(vector_to_center.y, vector_to_center.x) # In radians
    deg_angle = rad * (180 / 3.14159)
    
    real_angle = math.floor(deg_angle)
    distance_to_center = math.sqrt(vector_to_center.x * vector_to_center.x + vector_to_center.y * vector_to_center.y)
    #print(distance_to_center)
    
    positions.append(env.car.hull.position)
    #inner_loop[real_angle + 180] = round(distance_to_center, 2)
    xs.append(env.car.hull.position.x)
    ys.append(env.car.hull.position.y)
    
    real_angle = min(max(real_angle, -180), 359)
    if inner_loop[real_angle + 180] > round(distance_to_center, 2):
        print("left track inside at " + str(real_angle + 180)) 
    if outer_loop[real_angle + 180] < round(distance_to_center, 2):
        print("left track outside at " + str(real_angle + 180)) 

    #show image
    env.render()
    #action = env.action_space.sample()
    action = choosePlayerActionDiscrete()
    #print(str(type(action)))
    actions += str(action)
    env.step(action)
env.close()
print("start")
print(actions)
print("finished")

#for i in range(360):
#    if inner_loop[i] > outer_loop[i]:
#        print(i)
#        print("NOOOOO")
        
        
print("ok")
#print(inner_loop)

#fig, ax = plt.subplots()
#ax.scatter(xs, ys)
#ax.set_xlabel('x')
#ax.set_ylabel('y')
#ax.set_title("map")
#ax = plt.gca()

#plt.show()


Note: you may need to restart the kernel to use updated packages.


ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


start
0000000000000000000000000000000000000000000000000000000000000000000000000000000333333333333333333333333333333333333000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000222222222222000000022222222000000000022000022000000000000000000000000000000000000000000000000000000000000000000010000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000033333333333330000000000000000000033330000000000000000000000000000000000000000000000440000000000000000000033000000000020000002222000000200022200222200000000000000000000002222000000222222000000002222220000001100010000000011000001100003333333333000000220000000000022000000110000020000000033033033000000000200000000000002200000002200002200000000000000000000000000220000000000000022000000000000000000000001111000000001100100011100110000110011100011100000000110000000110000000000000000002222200022220002000022220220000